In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns',None)

    Select the batting team from the dropdown menu.
    Select the bowling team from the dropdown menu.
    Select the host city from the dropdown menu.
    Enter the target score.
    Enter the current score, overs completed, and wickets out.
    Click on the "Predict Probability" button to see the predicted win probability for each team.


     IPL First Innings Score Prediction

    IPL is very popular in India. The match of ipl has 20-20 overs, if we can estimate the score of the first innings. Although it is difficult to predict what will happen in cricket, this project would give us an estimate based on past matches. 

    Features
    Select the batting team from the dropdown menu.
    Select the bowling team from the dropdown menu.
    Select the overs greater than or equal to 5
    Select the venue
    Enter the  current score
    Enter the current wicket
    Enter runs scored in previous 5 overs
    Enter wickets taken in previous 5 overs
    Click on the "Predict Probability" button to see the predicted first inning score.


In [2]:
df_auction= pd.read_csv(r"E:\Github Files\ipl\IPL_Prediction\auction.csv")
df_delivery = pd.read_csv(r"E:\Github Files\ipl\IPL_Prediction\deliveries.csv")
df_ipl = pd.read_csv(r"E:\Github Files\ipl\IPL_Prediction\ipl.csv")
df_ball = pd.read_csv(r"E:\Github Files\ipl\IPL_Prediction\IPL_Ball_by_Ball_2008_2022.csv")
df_matches= pd.read_csv(r"E:\Github Files\ipl\IPL_Prediction\matches.csv")
df_ipl_matches = pd.read_csv(r"E:\Github Files\ipl\IPL_Prediction\IPL_Matches_2008_2022.csv")

In [3]:
# estimate the score of the first innings
df_delivery = df_delivery[df_delivery['inning'] == 1]

In [4]:
df_matches.head()

,id,Season,city,date,team1,team2,toss_winner,toss_decision,result,dl_applied,winner,win_by_runs,win_by_wickets,player_of_match,venue,umpire1,umpire2,umpire3
0,1,IPL-2017,Hyderabad,05-04-2017,Sunrisers Hyderabad,Royal Challengers Bangalore,Royal Challengers Bangalore,field,normal,0,Sunrisers Hyderabad,35,0,Yuvraj Singh,"Rajiv Gandhi International Stadium, Uppal",AY Dandekar,NJ Llong,NaN
1,2,IPL-2017,Pune,06-04-2017,Mumbai Indians,Rising Pune Supergiant,Rising Pune Supergiant,field,normal,0,Rising Pune Supergiant,0,7,SPD Smith,Maharashtra Cricket Association Stadium,A Nand Kishore,S Ravi,NaN
2,3,IPL-2017,Rajkot,07-04-2017,Gujarat Lions,Kolkata Knight Riders,Kolkata Knight Riders,field,normal,0,Kolkata Knight Riders,0,10,CA Lynn,Saurashtra Cricket Association Stadium,Nitin Menon,CK Nandan,NaN
3,4,IPL-2017,Indore,08-04-2017,Rising Pune Supergiant,Kings XI Punjab,Kings XI Punjab,field,normal,0,Kings XI Punjab,0,6,GJ Maxwell,Holkar Cricket Stadium,AK Chaudhary,C Shamshuddin,NaN
4,5,IPL-2017,Bangalore,08-04-2017,Royal Challengers Bangalore,Delhi Daredevils,Royal Challengers Bangalore,bat,normal,0,Royal Challengers Bangalore,15,0,KM Jadhav,M Chinnaswamy Stadium,NaN,NaN,NaN


In [5]:
df_delivery.head()

,match_id,inning,batting_team,bowling_team,over,ball,batsman,non_striker,bowler,is_super_over,wide_runs,bye_runs,legbye_runs,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs,player_dismissed,dismissal_kind,fielder
0,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,1,DA Warner,S Dhawan,TS Mills,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN
1,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,2,DA Warner,S Dhawan,TS Mills,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN
2,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,3,DA Warner,S Dhawan,TS Mills,0,0,0,0,0,0,4,0,4,NaN,NaN,NaN
3,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,4,DA Warner,S Dhawan,TS Mills,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN
4,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,5,DA Warner,S Dhawan,TS Mills,0,2,0,0,0,0,0,2,2,NaN,NaN,NaN


In [6]:
df_ipl.head()

,mid,date,venue,bat_team,bowl_team,batsman,bowler,runs,wickets,overs,runs_last_5,wickets_last_5,striker,non-striker,total
0,1,2008-04-18,M Chinnaswamy Stadium,Kolkata Knight Riders,Royal Challengers Bangalore,SC Ganguly,P Kumar,1,0,0.1,1,0,0,0,222
1,1,2008-04-18,M Chinnaswamy Stadium,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,P Kumar,1,0,0.2,1,0,0,0,222
2,1,2008-04-18,M Chinnaswamy Stadium,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,P Kumar,2,0,0.2,2,0,0,0,222
3,1,2008-04-18,M Chinnaswamy Stadium,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,P Kumar,2,0,0.3,2,0,0,0,222
4,1,2008-04-18,M Chinnaswamy Stadium,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,P Kumar,2,0,0.4,2,0,0,0,222


    Select the batting team from the dropdown menu.
    Select the bowling team from the dropdown menu.
    Select the overs greater than or equal to 5
    Select the venue
    Enter the  current score
    Enter the current wicket
    Enter runs scored in previous 5 overs
    Enter wickets taken in previous 5 overs
    Click on the "Predict Probability" button to see the predicted first inning score.


In [7]:
df_delivery.columns

Index(['match_id', 'inning', 'batting_team', 'bowling_team', 'over', 'ball',
       'batsman', 'non_striker', 'bowler', 'is_super_over', 'wide_runs',
       'bye_runs', 'legbye_runs', 'noball_runs', 'penalty_runs',
       'batsman_runs', 'extra_runs', 'total_runs', 'player_dismissed',
       'dismissal_kind', 'fielder'],
      dtype='object')

In [8]:
df_matches.columns

Index(['id', 'Season', 'city', 'date', 'team1', 'team2', 'toss_winner',
       'toss_decision', 'result', 'dl_applied', 'winner', 'win_by_runs',
       'win_by_wickets', 'player_of_match', 'venue', 'umpire1', 'umpire2',
       'umpire3'],
      dtype='object')

In [9]:
df_ipl.columns

Index(['mid', 'date', 'venue', 'bat_team', 'bowl_team', 'batsman', 'bowler',
       'runs', 'wickets', 'overs', 'runs_last_5', 'wickets_last_5', 'striker',
       'non-striker', 'total'],
      dtype='object')

In [10]:
df_matches_copy = df_matches[['id','venue','city']]
df_delivery_copy = df_delivery[['match_id','batting_team','bowling_team','total_runs']]
df_ipl_copy = df_ipl[['mid','runs_last_5','wickets_last_5','wickets','runs','overs','total']]

In [11]:
df_matches_merge = pd.merge(df_matches_copy,df_delivery_copy,left_on = 'id',right_on='match_id')

In [12]:
df = pd.merge(df_matches_merge,df_ipl_copy,left_on='id',right_on='mid')

In [13]:
df_matches_merge.duplicated().sum()

88478

In [14]:
df.duplicated().sum()

8899333

In [15]:
df1 = df.drop(['id','match_id','mid'],axis=1)

In [16]:
df1.duplicated().sum()

8899364

In [17]:
df1 = df1.drop_duplicates()

In [18]:
df1.shape

(428309, 11)

# Data Cleaning

In [19]:
def data_cleaning(df1):

    df1.replace(['Mumbai Indians','Kolkata Knight Riders','Royal Challengers Bangalore','Deccan Chargers','Chennai Super Kings',
                     'Rajasthan Royals','Delhi Daredevils','Gujarat Lions','Kings XI Punjab',
                     'Sunrisers Hyderabad','Rising Pune Supergiants','Kochi Tuskers Kerala','Pune Warriors','Rising Pune Supergiant']
                    ,['MI','KKR','RCB','DC','CSK','RR','DD','GL','KXIP','SRH','RPS','KTK','PW','RPS'],inplace=True)


data_cleaning(df1)  

In [20]:
df1 = df1.rename({'Maharashtra Cricket Association Stadium':"Subrata Roy Sahara Stadium"},axis=1)

In [21]:
df1.head()

,venue,city,batting_team,bowling_team,total_runs,runs_last_5,wickets_last_5,wickets,runs,overs,total
0,"Rajiv Gandhi International Stadium, Uppal",Hyderabad,SRH,RCB,0,1,0,0,1,0.1,222
1,"Rajiv Gandhi International Stadium, Uppal",Hyderabad,SRH,RCB,0,1,0,0,1,0.2,222
2,"Rajiv Gandhi International Stadium, Uppal",Hyderabad,SRH,RCB,0,2,0,0,2,0.2,222
3,"Rajiv Gandhi International Stadium, Uppal",Hyderabad,SRH,RCB,0,2,0,0,2,0.3,222
4,"Rajiv Gandhi International Stadium, Uppal",Hyderabad,SRH,RCB,0,2,0,0,2,0.4,222


In [22]:
df1.columns

Index(['venue', 'city', 'batting_team', 'bowling_team', 'total_runs',
       'runs_last_5', 'wickets_last_5', 'wickets', 'runs', 'overs', 'total'],
      dtype='object')

In [23]:
df1.shape

(428309, 11)

In [24]:
df1 = df1[df1['overs']>5]

In [25]:
X = df1.drop('total',axis=1)
y = df1['total']

In [26]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25,random_state=0)

In [27]:
print(X_train.shape,y_train.shape)
print(X_test.shape,y_test.shape)

(239574, 10) (239574,)
(79858, 10) (79858,)


In [28]:
# def data_encoding()

from sklearn.preprocessing import OneHotEncoder,StandardScaler,MinMaxScaler
from sklearn.neighbors import KNeighborsRegressor

In [29]:
X_train_cat = X_train.select_dtypes(include=['object'])
X_test_cat = X_test.select_dtypes(include=['object'])

X_train_num = X_train.select_dtypes(include=['int64','float64'])
X_test_num = X_test.select_dtypes(include=['int64','float64'])

In [30]:
X_train_cat.columns

Index(['venue', 'city', 'batting_team', 'bowling_team'], dtype='object')

In [31]:
X_train_cat['venue'].unique()

array(['M Chinnaswamy Stadium', 'Holkar Cricket Stadium',
       'Punjab Cricket Association IS Bindra Stadium, Mohali',
       'Wankhede Stadium', 'Rajiv Gandhi International Stadium, Uppal',
       'Brabourne Stadium', 'MA Chidambaram Stadium, Chepauk',
       'Dubai International Cricket Stadium', 'Kingsmead',
       'Sharjah Cricket Stadium', 'Dr DY Patil Sports Academy',
       'New Wanderers Stadium', 'Saurashtra Cricket Association Stadium',
       'Barabati Stadium', "St George's Park",
       'Punjab Cricket Association Stadium, Mohali',
       'Sawai Mansingh Stadium', 'Sardar Patel Stadium, Motera',
       'Eden Gardens', 'Feroz Shah Kotla', 'SuperSport Park',
       'De Beers Diamond Oval', 'Subrata Roy Sahara Stadium',
       'Sheikh Zayed Stadium', 'Newlands',
       'Vidarbha Cricket Association Stadium, Jamtha',
       'Dr. Y.S. Rajasekhara Reddy ACA-VDCA Cricket Stadium',
       'Maharashtra Cricket Association Stadium',
       'Himachal Pradesh Cricket Association Sta

In [32]:
X_train_cat['city'].unique()

array(['Bangalore', 'Indore', 'Chandigarh', 'Mumbai', 'Hyderabad',
       'Chennai', nan, 'Durban', 'Sharjah', 'Johannesburg', 'Rajkot',
       'Cuttack', 'Port Elizabeth', 'Jaipur', 'Ahmedabad', 'Kolkata',
       'Delhi', 'Centurion', 'Kimberley', 'Pune', 'Abu Dhabi',
       'Cape Town', 'Nagpur', 'Visakhapatnam', 'Dharamsala',
       'Bloemfontein', 'Ranchi', 'Kochi', 'East London', 'Raipur',
       'Kanpur'], dtype=object)

In [33]:
X_train_cat['batting_team'].unique()

array(['RCB', 'RR', 'DD', 'KXIP', 'DC', 'SRH', 'MI', 'KKR', 'CSK', 'GL',
       'PW', 'RPS', 'KTK'], dtype=object)

In [34]:
X_train_cat['bowling_team'].unique()

array(['MI', 'KTK', 'KXIP', 'CSK', 'KKR', 'DC', 'RCB', 'DD', 'GL', 'PW',
       'RR', 'RPS', 'SRH'], dtype=object)

In [35]:
a=X_train.sort_values(by='venue',ascending=True)

In [36]:
a

,venue,city,batting_team,bowling_team,total_runs,runs_last_5,wickets_last_5,wickets,runs,overs
2817478,Barabati Stadium,Cuttack,DC,KXIP,4,52,1,5,181,19.5
7500356,Barabati Stadium,Cuttack,MI,KKR,4,36,3,3,41,6.2
7500746,Barabati Stadium,Cuttack,MI,KKR,3,20,2,4,55,9.4
7420033,Barabati Stadium,Cuttack,KXIP,KKR,6,59,0,1,160,15.6
5095660,Barabati Stadium,Cuttack,DC,KKR,4,33,1,1,36,6.2
...,...,...,...,...,...,...,...,...,...,...
7667423,Wankhede Stadium,Mumbai,MI,DD,1,39,1,1,44,6.3
4687404,Wankhede Stadium,Mumbai,PW,MI,2,39,2,4,130,18.2
8706795,Wankhede Stadium,Mumbai,MI,RPS,4,42,1,3,106,13.2
1729635,Wankhede Stadium,Mumbai,RR,DD,6,16,2,2,71,9.1


In [37]:
a['venue'].unique()

array(['Barabati Stadium', 'Brabourne Stadium', 'Buffalo Park',
       'De Beers Diamond Oval', 'Dr DY Patil Sports Academy',
       'Dr. Y.S. Rajasekhara Reddy ACA-VDCA Cricket Stadium',
       'Dubai International Cricket Stadium', 'Eden Gardens',
       'Feroz Shah Kotla', 'Green Park',
       'Himachal Pradesh Cricket Association Stadium',
       'Holkar Cricket Stadium', 'JSCA International Stadium Complex',
       'Kingsmead', 'M Chinnaswamy Stadium',
       'MA Chidambaram Stadium, Chepauk',
       'Maharashtra Cricket Association Stadium', 'Nehru Stadium',
       'New Wanderers Stadium', 'Newlands', 'OUTsurance Oval',
       'Punjab Cricket Association IS Bindra Stadium, Mohali',
       'Punjab Cricket Association Stadium, Mohali',
       'Rajiv Gandhi International Stadium, Uppal',
       'Sardar Patel Stadium, Motera',
       'Saurashtra Cricket Association Stadium', 'Sawai Mansingh Stadium',
       'Shaheed Veer Narayan Singh International Stadium',
       'Sharjah Cricket S

In [38]:
from sklearn.preprocessing import LabelEncoder

In [39]:
def label_encoding(train_df,test_df):
    label_encoder = LabelEncoder()
    for col in train_df:
        train_df[col] = label_encoder.fit_transform(train_df[col])
        test_df[col] = label_encoder.transform(test_df[col])
        
        return train_df,test_df
X_train_transform, X_test_transform = label_encoding(X_train_cat,X_test_cat)

In [40]:
X_train_transform['venue'].unique()

array([14, 11, 21, 34, 23,  1, 15,  6, 13, 28,  4, 18, 25,  0, 30, 22, 26,
       24,  7,  8, 32,  3, 31, 29, 19, 33,  5, 16, 10, 20, 12, 17,  2, 27,
        9])

In [41]:
X_train_transform.sort_values(by='venue')

,venue,city,batting_team,bowling_team
2817478,0,Cuttack,DC,KXIP
7500356,0,Cuttack,MI,KKR
7500746,0,Cuttack,MI,KKR
7420033,0,Cuttack,KXIP,KKR
5095660,0,Cuttack,DC,KKR
...,...,...,...,...
7667423,34,Mumbai,MI,DD
4687404,34,Mumbai,PW,MI
8706795,34,Mumbai,MI,RPS
1729635,34,Mumbai,RR,DD


In [42]:
X_train_cat['venue']

9320003    14
4451465    11
518382     21
1741951    34
5230585    34
           ..
3567789     4
8860004    14
4435120    10
3433036    14
8906401    34
Name: venue, Length: 239574, dtype: int32

In [43]:
X_test_transform

,venue,city,batting_team,bowling_team
4410851,14,Bangalore,KKR,RCB
1759655,4,Mumbai,CSK,RR
7742392,34,Mumbai,RR,MI
7213950,6,NaN,SRH,MI
2481602,18,Johannesburg,KXIP,DC
...,...,...,...,...
5065730,8,Delhi,PW,DD
8710586,34,Mumbai,MI,RPS
4759401,31,Pune,PW,KXIP
500700,25,Rajkot,GL,MI


In [44]:
encoder = OneHotEncoder(drop='first',sparse_output=False)
X_train_transfromed = pd.DataFrame(encoder.fit_transform(X_train_cat),
columns = encoder.get_feature_names_out(X_train_cat.columns),
                                  index=X_train_cat.index)

In [45]:
X_test_transformed = pd.DataFrame(encoder.transform(X_test_cat),
                                 columns = encoder.get_feature_names_out(X_test_cat.columns),
                                 index = X_test_cat.index)

In [46]:
X_test_transformed

,venue_1,venue_2,venue_3,venue_4,venue_5,venue_6,venue_7,venue_8,venue_9,venue_10,venue_11,venue_12,venue_13,venue_14,venue_15,venue_16,venue_17,venue_18,venue_19,venue_20,venue_21,venue_22,venue_23,venue_24,venue_25,venue_26,venue_27,venue_28,venue_29,venue_30,venue_31,venue_32,venue_33,venue_34,city_Ahmedabad,city_Bangalore,city_Bloemfontein,city_Cape Town,city_Centurion,city_Chandigarh,city_Chennai,city_Cuttack,city_Delhi,city_Dharamsala,city_Durban,city_East London,city_Hyderabad,city_Indore,city_Jaipur,city_Johannesburg,city_Kanpur,city_Kimberley,city_Kochi,city_Kolkata,city_Mumbai,city_Nagpur,city_Port Elizabeth,city_Pune,city_Raipur,city_Rajkot,city_Ranchi,city_Sharjah,city_Visakhapatnam,city_nan,batting_team_DC,batting_team_DD,batting_team_GL,batting_team_KKR,batting_team_KTK,batting_team_KXIP,batting_team_MI,batting_team_PW,batting_team_RCB,batting_team_RPS,batting_team_RR,batting_team_SRH,bowling_team_DC,bowling_team_DD,bowling_team_GL,bowling_team_KKR,bowling_team_KTK,bowling_team_KXIP,bowling_team_MI,bowling_team_PW,bowling_team_RCB,bowling_team_RPS,bowling_team_RR,bowling_team_SRH
4410851,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1759655,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
7742392,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
7213950,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2481602,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5065730,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8710586,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4759401,0.0,0

In [47]:
88+10

98

In [48]:
X_train_transfromed.columns

Index(['venue_1', 'venue_2', 'venue_3', 'venue_4', 'venue_5', 'venue_6',
       'venue_7', 'venue_8', 'venue_9', 'venue_10', 'venue_11', 'venue_12',
       'venue_13', 'venue_14', 'venue_15', 'venue_16', 'venue_17', 'venue_18',
       'venue_19', 'venue_20', 'venue_21', 'venue_22', 'venue_23', 'venue_24',
       'venue_25', 'venue_26', 'venue_27', 'venue_28', 'venue_29', 'venue_30',
       'venue_31', 'venue_32', 'venue_33', 'venue_34', 'city_Ahmedabad',
       'city_Bangalore', 'city_Bloemfontein', 'city_Cape Town',
       'city_Centurion', 'city_Chandigarh', 'city_Chennai', 'city_Cuttack',
       'city_Delhi', 'city_Dharamsala', 'city_Durban', 'city_East London',
       'city_Hyderabad', 'city_Indore', 'city_Jaipur', 'city_Johannesburg',
       'city_Kanpur', 'city_Kimberley', 'city_Kochi', 'city_Kolkata',
       'city_Mumbai', 'city_Nagpur', 'city_Port Elizabeth', 'city_Pune',
       'city_Raipur', 'city_Rajkot', 'city_Ranchi', 'city_Sharjah',
       'city_Visakhapatnam', 'city_nan'

In [49]:
X_train_transform = pd.concat([X_train_num,X_train_transfromed],axis=1)


In [50]:
X_test_transfrom = pd.concat([X_test_num,X_test_transformed],axis=1)

In [51]:
X_train_transform.head()

,total_runs,runs_last_5,wickets_last_5,wickets,runs,overs,venue_1,venue_2,venue_3,venue_4,venue_5,venue_6,venue_7,venue_8,venue_9,venue_10,venue_11,venue_12,venue_13,venue_14,venue_15,venue_16,venue_17,venue_18,venue_19,venue_20,venue_21,venue_22,venue_23,venue_24,venue_25,venue_26,venue_27,venue_28,venue_29,venue_30,venue_31,venue_32,venue_33,venue_34,city_Ahmedabad,city_Bangalore,city_Bloemfontein,city_Cape Town,city_Centurion,city_Chandigarh,city_Chennai,city_Cuttack,city_Delhi,city_Dharamsala,city_Durban,city_East London,city_Hyderabad,city_Indore,city_Jaipur,city_Johannesburg,city_Kanpur,city_Kimberley,city_Kochi,city_Kolkata,city_Mumbai,city_Nagpur,city_Port Elizabeth,city_Pune,city_Raipur,city_Rajkot,city_Ranchi,city_Sharjah,city_Visakhapatnam,city_nan,batting_team_DC,batting_team_DD,batting_team_GL,batting_team_KKR,batting_team_KTK,batting_team_KXIP,batting_team_MI,batting_team_PW,batting_team_RCB,batting_team_RPS,batting_team_RR,batting_team_SRH,bowling_team_DC,bowling_team_DD,bowling_team_GL,bowling_team_KKR,bowling_team_KTK,bowling_team_KXIP,bowling_team_MI,bowling_team_PW,bowling_team_RCB,bowling_team_RPS,bowling_team_RR,bowling_team_SRH
9320003,2,30,2,5,76,12.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4451465,4,37,1,2,124,14.3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
518382,4,44,2,3,58,7.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1741951,2,30,0,4,83,13.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5230585,0,50,2,6,133,18.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [52]:
X_train_transform.columns

Index(['total_runs', 'runs_last_5', 'wickets_last_5', 'wickets', 'runs',
       'overs', 'venue_1', 'venue_2', 'venue_3', 'venue_4', 'venue_5',
       'venue_6', 'venue_7', 'venue_8', 'venue_9', 'venue_10', 'venue_11',
       'venue_12', 'venue_13', 'venue_14', 'venue_15', 'venue_16', 'venue_17',
       'venue_18', 'venue_19', 'venue_20', 'venue_21', 'venue_22', 'venue_23',
       'venue_24', 'venue_25', 'venue_26', 'venue_27', 'venue_28', 'venue_29',
       'venue_30', 'venue_31', 'venue_32', 'venue_33', 'venue_34',
       'city_Ahmedabad', 'city_Bangalore', 'city_Bloemfontein',
       'city_Cape Town', 'city_Centurion', 'city_Chandigarh', 'city_Chennai',
       'city_Cuttack', 'city_Delhi', 'city_Dharamsala', 'city_Durban',
       'city_East London', 'city_Hyderabad', 'city_Indore', 'city_Jaipur',
       'city_Johannesburg', 'city_Kanpur', 'city_Kimberley', 'city_Kochi',
       'city_Kolkata', 'city_Mumbai', 'city_Nagpur', 'city_Port Elizabeth',
       'city_Pune', 'city_Raipur', 'ci

In [53]:
X_train_transform.head(2)

,total_runs,runs_last_5,wickets_last_5,wickets,runs,overs,venue_1,venue_2,venue_3,venue_4,venue_5,venue_6,venue_7,venue_8,venue_9,venue_10,venue_11,venue_12,venue_13,venue_14,venue_15,venue_16,venue_17,venue_18,venue_19,venue_20,venue_21,venue_22,venue_23,venue_24,venue_25,venue_26,venue_27,venue_28,venue_29,venue_30,venue_31,venue_32,venue_33,venue_34,city_Ahmedabad,city_Bangalore,city_Bloemfontein,city_Cape Town,city_Centurion,city_Chandigarh,city_Chennai,city_Cuttack,city_Delhi,city_Dharamsala,city_Durban,city_East London,city_Hyderabad,city_Indore,city_Jaipur,city_Johannesburg,city_Kanpur,city_Kimberley,city_Kochi,city_Kolkata,city_Mumbai,city_Nagpur,city_Port Elizabeth,city_Pune,city_Raipur,city_Rajkot,city_Ranchi,city_Sharjah,city_Visakhapatnam,city_nan,batting_team_DC,batting_team_DD,batting_team_GL,batting_team_KKR,batting_team_KTK,batting_team_KXIP,batting_team_MI,batting_team_PW,batting_team_RCB,batting_team_RPS,batting_team_RR,batting_team_SRH,bowling_team_DC,bowling_team_DD,bowling_team_GL,bowling_team_KKR,bowling_team_KTK,bowling_team_KXIP,bowling_team_MI,bowling_team_PW,bowling_team_RCB,bowling_team_RPS,bowling_team_RR,bowling_team_SRH
9320003,2,30,2,5,76,12.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4451465,4,37,1,2,124,14.3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Algorithms:-

1. Linear Regression
2. Support Vector Regression
3. Decision Tree Regression
4. Random Forest Regression 
5. Artificial Neural Network
4. Recursive Neural Network

In [60]:
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler,MinMaxScaler
import Pipelines

ModuleNotFoundError: No module named 'Pipelines'

In [55]:
import time 
import os

In [56]:


param_grids = {
    'linear':[
        {
            'scaler':[StandardScaler(),MinMaxScaler()]
        }
    ],
    'svr':[
        {
            'scaler':[StandardScaler(),MinMaxScaler()],
            'regression__kernel':['linear'],
            'regression__C':[0.1,0.001,1,10,100]
        },
        {
            'scaler':[StandardScaler(),MinMaxScaler()],
            'regression__kernel':['poly'],
            'regression__C':[0.1,0.001,1,10,100]
        },
        {
            'scaler': [StandardScaler(), MinMaxScaler()],
            'regression__kernel' : ['rbf'], 
            'regression__C' : [0.1, 0.01, 1, 10, 100]
        }
          
    ],
    'decision_tree':[
        {
            'scaler':[StandardScaler(),MinMaxScaler()],
            'regression__max_depth':[i for i in range(1,200,3)],
            'regression__max_leaf_nodes':[i for i in range(2,10)]
        }
    ],
    'random_forest':[
        {
            'scaler':[StandardScaler(),MinMaxScaler()],
            'regression__min_samples_split': [i for i in range(2, 10)],
            'regression__max_leaf_nodes':[i for i in range(2,20)]
       
        }
    ]
    
}

best_models={}

for algo in pipelines.keys():
    print(algo)
    grid_search = GridSearchCV(estimator=pipelines[algo],
                              param_grid=param_grids[algo],
                              cv=5,
                              return_train_score=True,
                              verbose=1)
    
# fit 
start_fit_time = time.time()
grid_search.fit(X_train_transform,y_train)
end_fit_time = time.time()

#predict 
start_predict_time = time.time()
y_pred = grid_search.predict(X_test_transfrom)
end_predict_time = time.time()

#saving the best models
joblib.dump(grid_search.best_estimator_,f'best_model/{algo}.pkl')
model_size = os.path.getsize(f'best_models/{algo}.pkl')

#print*
print('Train Score: ', grid_search.best_score_)
print('Test Score: ', grid_search.score(X_test, y_test))
print("Fit Time: ", end_fit_time - start_fit_time)
print("Predict Time: ", end_predict_time - start_predict_time)
print("Model Size: ", model_size)

In [57]:
best_models={}

for algo in pipelines.keys():
    print(algo)
    grid_search = GridSearchCV(estimator=pipelines[algo],
                              param_grid=param_grids[algo],
                              cv=5,
                              return_train_score=True,
                              verbose=1)
    
    # Fit
    start_fit_time = time.time()
    grid_search.fit(X_train_transform, y_train)
    end_fit_time = time.time()

    # Predict
    start_predict_time = time.time()
    y_pred = grid_search.predict(X_test_transform)
    end_predict_time = time.time()

    # Saving the best models
    joblib.dump(grid_search.best_estimator_, f'best_model/{algo}.pkl')
    model_size = os.path.getsize(f'best_models/{algo}.pkl')

    # Evaluation metrics
    train_score = grid_search.best_score_
    test_score = grid_search.score(X_test_transform, y_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    rmse = sqrt(mse)

    # Print evaluation metrics
    print('Train Score: ', train_score)
    print('Test Score: ', test_score)
    print('Mean Squared Error (MSE): ', mse)
    print('R2 Score: ', r2)
    print('Root Mean Squared Error (RMSE): ', rmse)
    print("Fit Time: ", end_fit_time - start_fit_time)
    print("Predict Time: ", end_predict_time - start_predict_time)
    print("Model Size: ", model_size)

    # Plot predictions vs actual values
    plt.scatter(y_test, y_pred)
    plt.xlabel('Actual')
    plt.ylabel('Predicted')
    plt.title(f'{algo} Predictions vs Actual')
    plt.show()

NameError: name 'pipelines' is not defined

In [ ]:
# # Define pipelines
# pipelines = {
#     'linear': Pipeline([
#         ('scaler', StandardScaler()),
#         ('regression', LinearRegression())
#     ]),
#     'svr': Pipeline([
#         ('scaler', StandardScaler()),
#         ('regression', SVR())
#     ]),
#     'decision_tree': Pipeline([
#         ('scaler', StandardScaler()),
#         ('regression', DecisionTreeRegressor())
#     ]),
#     'random_forest': Pipeline([
#         ('scaler', StandardScaler()),
#         ('regression', RandomForestRegressor())
#     ])
# }

# # Define parameter grids
# param_grids = {
#     'linear': [
#         {'scaler': [StandardScaler(), MinMaxScaler()]}
#     ],
#     'svr': [
#         {'scaler': [StandardScaler(), MinMaxScaler()],
#          'regression__kernel': ['linear'],
#          'regression__C': [0.1, 0.001, 1, 10, 100]},
#         {'scaler': [StandardScaler(), MinMaxScaler()],
#          'regression__kernel': ['poly'],
#          'regression__C': [0.1, 0.001, 1, 10, 100]},
#         {'scaler': [StandardScaler(), MinMaxScaler()],
#          'regression__kernel': ['rbf'],
#          'regression__C': [0.1, 0.01, 1, 10, 100]}
#     ],
#     'decision_tree': [
#         {'scaler': [StandardScaler(), MinMaxScaler()],
#          'regression__max_depth': [i for i in range(1, 200, 3)],
#          'regression__max_leaf_nodes': [i for i in range(2, 10)]}
#     ],
#     'random_forest': [
#         {'scaler': [StandardScaler(), MinMaxScaler()],
#          'regression__min_samples_split': [i for i in range(2, 10)],
#          'regression__max_leaf_nodes': [i for i in range(2, 20)]}
#     ]
# }

# best_models = {}

# # Train and save best models
# for algo in pipelines.keys():
#     print(algo)
#     grid_search = GridSearchCV(estimator=pipelines[algo],
#                                param_grid=param_grids[algo],
#                                cv=5,
#                                return_train_score=True,
#                                verbose=1)
    
#     # Fit
#     start_fit_time = time.time()
#     grid_search.fit(X_train, y_train)
#     end_fit_time = time.time()

#     # Predict
#     start_predict_time = time.time()
#     y_pred = grid_search.predict(X_test)
#     end_predict_time = time.time()

#     # Saving the best models
#     joblib.dump(grid_search.best_estimator_, f'best_models/{algo}.pkl')
#     model_size = os.path.getsize(f'best_models/{algo}.pkl')

#     # Print
#     print('Train Score: ', grid_search.best_score_)
#     print('Test Score: ', grid_search.score(X_test, y_test))
#     print("Fit Time: ", end_fit_time - start_fit_time)
#     print("Predict Time: ", end_predict_time - start_predict_time)
#     print("Model Size: ", model_size)